In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from fitter import Fitter
from scipy.stats import exponnorm, erlang, gennorm


In [3]:
def load_data(column):
    stream = []
    days = range(1, 23) # 1, 32
    months = range(1, 2) #1, 3
    for month in months:
        month_2_digits = '{month:02}'.format(month=month)
        path = './COVID19_Tweets_Dataset_2020/Summary_Sentiment/2020_' + month_2_digits + '/'

        for day in days:
            if month == 2 and day > 29:
                continue
            if month == 1 and day < 22:
                continue

            for hour in range(24):
                file_name = path + f'2020_{month_2_digits}_' + '{day:02}'.format(day=day) + '_{hour:02}'.format(hour=hour) + '_Summary_Sentiment.csv'
                stream.append(pd.read_csv(file_name)[column])

    return pd.concat(stream, ignore_index=True), stream

In [4]:
stream_ln, tweets_per_file = load_data('Logits_Negative')

In [5]:
def get_fitted_ks_values(data):
    f = Fitter(data)
    f.fit()
    return f.summary(plot=False, clf=False)['ks_pvalue'].values

In [6]:
ks_values = []
for tweets_hour in tweets_per_file:
    ks_values.append(get_fitted_ks_values(tweets_hour))
    break

np.mean([i>=.01 for i in np.concatenate(ks_values)])

Fitting 110 distributions:  73%|███████▎  | 80/110 [00:15<00:18,  1.63it/s]SKIPPED rv_continuous distribution (taking more than 30 seconds)
SKIPPED rv_histogram distribution (taking more than 30 seconds)
Fitting 110 distributions:  98%|█████████▊| 108/110 [00:46<00:08,  4.10s/it]/home/thalis/repositories/TCC/venv/lib/python3.8/site-packages/scipy/integrate/_quadpack_py.py:1225: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,
SKIPPED vonmises distribution (taking more than 30 seconds)
Fitting 110 distributions: 100%

1.0

In [7]:
np.mean([i>=.01 for i in get_fitted_ks_values(stream_ln)])

Fitting 110 distributions:   1%|          | 1/110 [00:00<00:24,  4.47it/s]/home/thalis/repositories/TCC/venv/lib/python3.8/site-packages/scipy/integrate/_quadpack_py.py:1225: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,
Fitting 110 distributions:   6%|▋         | 7/110 [00:30<05:38,  3.29s/it]SKIPPED betaprime distribution (taking more than 30 seconds)
SKIPPED arcsine distribution (taking more than 30 seconds)
SKIPPED burr distribution (taking more than 30 seconds)
Fitting 110 distributions:  17%|█▋        | 19/110 [01:00<03:33,  2.34s/it]SKIPPED erlang distribution (taking more than 30 seconds)
SKIPPED exponpow distribution (taking more than 30 seconds)
Fitting 110 distributions:  72%|███████▏  | 79/110 [04:01<00:50,  1.63s/it]SKIPPED reciprocal distribution (taking more than 30 seconds)
SKIPPED ncx2 distribution (taking more than 30 seconds)
Fitting 110 distributions:  73%|███████▎

0.0